In [ ]:
! pip install yfinance

     |████████████████████████████████| 6.3MB 13.8MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=7f428ec56b3b8b4b8e71a4b343c64125fb038cd7b6fb3a140bb437d7cb14a938
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


# Importing necessary libraries

In [ ]:
from bs4 import BeautifulSoup as BS
import requests
import yfinance as yf
import datetime
import pandas as pd
import pandas_datareader.data as web


import urllib.request as urllib2 
from urllib.request import URLError

from tqdm import tqdm

import os 
import pickle
import numpy as np



# Getting the list of tickers

In [ ]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = BS(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]
start = datetime.datetime(2021,4,28)
end = datetime.datetime(2021,5,28)


# Getting market cap data for each ticker

In [ ]:
dates_keys = ['2021-04-28', '2021-04-29', '2021-04-30',  '2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06']
dates_keys = dates_keys + ['2021-05-07', '2021-05-10', '2021-05-11', '2021-05-12', '2021-05-13', '2021-05-14', '2021-05-17', '2021-05-18', '2021-05-19']
dates_keys = dates_keys + ['2021-05-20', '2021-05-21', '2021-05-24', '2021-05-25', '2021-05-26', '2021-05-27', '2021-05-28']
len(dates_keys)

23

# Try to upload data 

In [ ]:
if 'market_cap.pkl' in os.listdir('.'):
  with open('market_cap.pkl', 'rb') as file:
    market_cap_dict = pickle.load(file)

# If there is no data to upload -> download it yourself

In [ ]:
if 'market_cap.pkl' not in os.listdir('.'):

  market_cap_dict = {}

  for ticker in tqdm(tickers):
    soup = BS(urllib2.urlopen('https://ycharts.com/companies/' + ticker + '/market_cap'), features="html.parser")
    leftCol = soup.find('div', {'class':'dataColLeft'})
    dates = leftCol.find_all('td', {'class':'col1'})
    market_caps = leftCol.find_all('td', {'class':'col2'})

    dates = list(map(lambda x: x.string, dates))
    market_caps = list(map(lambda x: x.string.strip(), market_caps))

    zipped = list(zip(dates, market_caps))

    # preprocess the data
    dates = []
    for zipp in zipped:
      month_day, year = zipp[0].split(', ')
      month, day = month_day.split(' ')
      dates.append((year + ('-05-' if month == 'May' else '-04-') + ('0' if int(day) < 10 else '') + day))

    prices = []
    for zipp in zipped:
      value = int(zipp[1].replace('.', '').replace('T', '000000000').replace('B', '000000'))
      prices.append(value)

    marketCapKnown = dict(list(zip(dates, prices)))

    marketCap = dict([(dk, 0) for dk in dates_keys])
    marketCap.update(marketCapKnown)

    market_cap_dict[ticker] = marketCap


# Save data to reuse it later

In [ ]:
with open('market_cap.pkl', 'wb') as file:
  pickle.dump(market_cap_dict, file)

# Download yfinance data

In [ ]:
data = yf.download(tickers, start=start, end=end)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Get Adj Close data

In [ ]:
adj_close_data = data['Adj Close']

Calculating change in percentages

In [ ]:
ptc_changes = adj_close_data.pct_change()
ptc_changes = ptc_changes.dropna(how='all')
ptc_changes = ptc_changes.dropna(axis=1,how='all')
ptc_changes

In [ ]:
number_of_companies = 496
changes_of_prices = ptc_changes.sum(axis=1)

index_change_ptc = changes_of_prices/number_of_companies+1

In [ ]:
index_change_ptc_df=pd.DataFrame(index_change_ptc)

# Projecting and comparing the projection to the real values

In [ ]:
sp_curr=4183.18  # s&p index value on the starting date
sp_list=[sp_curr]
for ind in index_change_ptc:
  sp_curr=sp_curr*ind
  sp_list.append(sp_curr)

In [ ]:
SP500 = web.DataReader(['sp500'], 'fred', start, end)

In [ ]:
real_SP500 = SP500.dropna(axis=0,how='all')
real_SP500 = real_SP500[1:]

In [ ]:
sp_projection_df = pd.DataFrame(sp_list, index=real_SP500.index)

In [ ]:
comparison_ew=pd.concat([real_SP500,sp_projection_df],axis=1)
comparison_ew


,sp500,0
DATE,,
2021-04-29,4211.47,4183.180000
2021-04-30,4181.17,4213.192083
2021-05-03,4192.66,4179.263246
2021-05-04,4164.66,4204.885090
2021-05-05,4167.59,4197.809560
2021-05-06,4201.62,4209.994260
2021-05-07,4232.60,4231.108964
2021-05-10,4188.43,4276.933424
2021-05-11,4152.10,4258.474583


# Involving market capitalization

In [ ]:
market_cap_df = pd.DataFrame.from_dict(market_cap_dict)

market_cap_list = market_cap_df.values.tolist()
market_cap_sums = list(map(sum, market_cap_list))


for i in range(len(market_cap_df)):
  market_cap_df.iloc[i] = market_cap_df.iloc[i] / market_cap_sums[i]

# sort rows by date
market_cap_df = market_cap_df.sort_index().iloc[3:-1]
# sort columns by name
market_cap_df = market_cap_df.reindex(sorted(market_cap_df.columns), axis=1)
market_cap_df

# Do the dot product


In [ ]:
columns_for_removal = []
for mcd in market_cap_df.columns:
  if mcd not in ptc_changes.columns:
    columns_for_removal.append(mcd)

market_cap_df = market_cap_df.drop(columns_for_removal, axis=1)


In [ ]:
market_cap_list = market_cap_df.values.tolist()
ptc_change_list = ptc_changes.values.tolist()

lists = list(zip(market_cap_list, ptc_change_list))
weighted_change = list(map(lambda x: 1+sum(i[0] * i[1] for i in zip(x[0], x[1])), lists))
weighted_change


# Comparing outputs

In [ ]:
sp_curr=4183.18  # s&p index value on the starting date
sp_list=[sp_curr]
for ind in weighted_change:
  sp_curr=sp_curr*ind
  sp_list.append(sp_curr)

In [ ]:
weighted_sp_projection_df = pd.DataFrame(sp_list, index=real_SP500.index)

In [ ]:
comparison=pd.concat([ weighted_sp_projection_df, comparison_ew],axis=1)
comparison

In [ ]:
sum_sq_w = 0
sum_sq_e = 0

for triple in comparison.values.tolist():
  sum_sq_w += (triple[0] - triple[1])**2
  sum_sq_e += (triple[2] - triple[1])**2

length = len(comparison.values.tolist())
sum_sq_w /= length
sum_sq_e /= length

np.sqrt(sum_sq_w), np.sqrt(sum_sq_e)
  

(87.60982311553664, 60.18598892256564)